# Quantum Gases
### Authors: Lorenzo Braccini, Prof. Bart Hoogenboom, University College London

<font color=red>Note: the red texts are meant as side notes during the implementation.

</font> 

### Running instructions

The notebook runs from top to bottom and you should ``Run`` each input cell one after the other. 

## Introduction:

## Content of the notebook:
1. [Section 1](#1) **1D Density of States**
2. [Section 2](#2) **2D and 3D Densities of States**
3. [Section 3](#3) **Partition Functions**

In [11]:
%%capture
# IMPORTANT - run this cell to ensure videos work
!conda install -y -q ffmpeg 

In [12]:
# Imports

import numpy as np # advanced maths 
import matplotlib.pyplot as plt # plots 
from mpl_toolkits.mplot3d import Axes3D # 3D plots
import matplotlib.animation as animation # animation (video)
import matplotlib.gridspec as gridspec # fancy subplots
from IPython.display import HTML # shows animation in jupyter
from ipywidgets import interact, interactive, fixed, interact_manual # add interactions
from tqdm import tqdm_notebook as tqdm

In [13]:
matplotlib inline

## 1) <a id='1'> </a> 1D Density of States


To develop the statistical physics of a quantum gas, we need to count the quantum states of the particles. For this purpose, we consider a particle of mass $m$ in a box of length L. First, we will consider a one dimentional system, and later we will expand the problem to two and three dimensions.  

### 1-D Infinite Square Well
The wavefunction of a free particle (with null potential $V(x) = 0$) has the form

$$
\psi_k \propto \sin(kx)
$$
In a 1-D infinite square well , applying boundary conditions and normalization contrain, the Schrödinger wavefunction becomes:
$$
\psi_n (x)= \sqrt{\frac{2}{L}} \sin \left(\frac{n \pi}{L}x \right)
$$
and energy:
$$
E_n = \frac{\hbar^2 k^2}{2 m} = \frac{\hbar^2 n^2 \pi^2}{2 m L^2}
$$


In [14]:
def wavefunction1D(x, n, L, t, alpha):
    """
    Calculate the wavefunction of a particle in a box 1-D
    Inputs:
    x      Values of x
    n      Quantum number
    L      Lenght of the box
    alpha  h^2 pi^2/(2 m)    
    Output: the wavefunction
    """
    return np.sqrt(2/L)*np.sin(np.pi * n * x / L)*np.cos(alpha*n**2*t/L**2)

In [25]:
%%capture

L = 1         # Lenght of the box
alpha = 1     # h^2 pi^2/(2 m)     
dt = 100      # interval ms   
Nframe = 156  # Number of frames


n_max = 4
fig1 = plt.figure(figsize = (10,10)) # initialise the figure  
ax1 = fig1.add_subplot(111)
xs_1 = np.linspace(0, L, 50) #calulate x values
colors = ['b','b','k','g','m']

ax1.set_yticks(np.arange(- 1,n_max**2 + 7, 1))
tick_y = ax1.get_yticks().tolist()
ns = np.arange(1, n_max+1, 1)**2
n = 1
for i in range(len(tick_y)):
    if tick_y[i] in ns:
        tick_y[i] = r'$E{{ {0} }} $'.format(n) #r'$\frac{L}{\pi}$'
        n += 1
    else:
        tick_y[i] = ''

ax1.set_xticks(np.arange(0, L+0.1, 0.1))
tick_x = ax1.get_xticks().tolist()
ns = np.arange(1, n_max, 1)**2
n = 1
for i in range(len(tick_x)):
    if tick_x[i] == 1.0:
        tick_x[i] = 'L' #r'$\frac{L}{\pi}$'
    elif tick_x[i] == 0.0:
        tick_x[i] = '0' #r'$\frac{L}{\pi}$'
    else:
        tick_x[i] = ''

labels = [r'$\lambda_1 = 2 L \;\;\; k_1 = \frac{\pi}{L} \;\;\; E_1 = \frac{\hbar^2 \pi^2}{2 m L^2}$',
         r'$\lambda_2 = L \;\;\;\;\; k_2 = 2 \frac{\pi}{L} \;\;\; E_2 = 4 \frac{\hbar^2 \pi^2}{2 m L^2}$',
         r'$\lambda_3 = \frac{3}{2} L \;\;\; k_3 = 3 \frac{\pi}{L} \;\;\; E_3 = 9 \frac{\hbar^2 \pi^2}{2 m L^2} $',
         r'$\lambda_4 = \frac{L}{2} \;\;\;\; k_4 = 4 \frac{\pi}{L} \;\;\; E_4 = 16 \frac{\hbar^2 \pi^2}{2 m L^2} $']

def particle_box(frame):
    ax1.clear()
    t = frame*0.04 # time of the animation
    for n in range(1, n_max+1):
        ax1.plot(xs_1, n**2 + wavefunction1D(xs_1, n, L, t, alpha), color= colors[n], 
                label = labels[n-1])
        ax1.axhline(y=n**2, color='gray', alpha = 0.3)
    
    ax1.set_yticks(np.arange(- 1,n_max**2 + 7, 1))
    ax1.set_xticks(np.arange(0, L+0.1, 0.1))
    ax1.set_xlabel('x')
    ax1.set_ylabel('Energy')
    #ax1.grid()
    ax1.axvline(x=0, color='r')
    ax1.axvline(x=L, color='r')
    ax1.set_yticklabels(tick_y)
    ax1.set_xticklabels(tick_x)
    ax1.set_ylim(-0.5, n_max**2 + 7)
    ax1.legend(loc='upper center')
    ax1.set_title('1D Infinite Square Well');



In [26]:
# create the animation
ani1 = animation.FuncAnimation(fig1, particle_box, init_func=None, interval=dt, frames = tqdm(range(Nframe))) 
HTML(ani1.to_html5_video())

<font color=red>Note: I used the legend to put wavelength, k and energy, and not next at the right axes because it is hard to implement and quite computational heavy and it would take time both to implement and run. Let me know if this is fine. If not, I will try to implement it.

</font> 



### 1D Density of States

Given the quantised nature of $k$ (and thus of the energy), the states of a particle in a box represent a series a point in the "k-space" separated by $\Delta k = \frac{\pi}{L}$. *i.e.* every increment of $\Delta k = \frac{\pi}{L}$ contains one k-state. This give the density of states:
$$
\rho (k) = \frac{\pi}{L} 
$$
Since, in 1-D, each k states correspond to an energy state $\epsilon = \frac{\hbar^2 k^2}{2 m}$, then there is one state per $\Delta \epsilon = \Delta k \cdot \frac{\Delta \epsilon}{\Delta k}$, *i.e.*:
$$
\Delta \epsilon \approx \Delta k \cdot \frac{d \epsilon}{d k} = \frac{\pi}{L} \frac{\hbar^2 k}{2 m} \propto \sqrt{\epsilon}
$$
which gives the density of states:
$$
g(\epsilon) = \frac{1}{\Delta \epsilon} \propto \frac{1}{\sqrt{\epsilon}}
$$

In [27]:
n_max = 8
L = 1
alpha = 1

xs3_discrate = np.arange(1,n_max+1,1)
xs3 = np.linspace(0, n_max*1.05, 50)

@interact(n=(1,n_max, 1)) # interacting velocity 
def multiplicity(n):
    fig3 = plt.figure(figsize = (12,8),constrained_layout=True)
    grid = fig3.add_gridspec(3,2)
    
    ###### First subplot #######
    ax1 = fig3.add_subplot(grid[0,0])
    # plot the scatter points
    for i in range(0,n_max+1):
        if i == n:
            ax1.scatter(xs3_discrate[i-1], 0, color = 'r', s = 60, zorder=2)
        else:
            ax1.scatter(xs3_discrate[i-1], 0, color = 'b', s = 40, zorder=2)
    
    # ticks for the k axis 
    ax1.set_xticks(np.arange(0,n_max+1, 1))
    tick_k = ax1.get_xticks().tolist()
    for i in range(len(tick_k)):
        if i == 0:
            tick_k[i] = '0'
        else:
            tick_k[i] = r'$k_{{ {0} }}$'.format(int(tick_k[i]))
    
    # plot additional feature of the graph
    ax1.set_xlabel('$k$') 
    ax1.axhline(y=0, color='k', alpha = 0.8,  zorder=0)
    ax1.set_yticklabels([])
    ax1.set_xticklabels(tick_k)
    ax1.grid(axis = 'x')
    ax1.set_title('States in the k-space')
    rectangle = plt.Rectangle((n - 0.5,-2),1, 4, alpha = 0.4, color = 'r', zorder=1)
    ax1.add_artist(rectangle)
    
    ###### Second subplot #######
    ax2 = fig3.add_subplot(grid[0,1])
    
    # calculate the points of the plot
    xs2 = np.arange(1, n_max + 1, 1)
    ys2 = np.zeros(len(xs2))
    for i in range(len(xs2)):
        ys2[i] = np.sum(np.where((xs3_discrate >= xs2[i]) & (xs3_discrate < xs2[i]+1) , 1, 0))
        
    for i in range(len(xs2)):
        if xs2[i] == n:
            ax2.scatter(xs2[i], ys2[i], color = 'r', s = 60, zorder=2,)
        else:
            ax2.scatter(xs2[i], ys2[i], color = 'b', s = 40, zorder=2,)
            
    ax2.axhline(y=1, color='k', alpha = 0.8,  zorder=0, label=r'$\rho(k) = 1$')
    
    # plot additional feature of the graph
    ax2.set_xticks(np.arange(0,n_max+1, 1))
    ax2.set_xticklabels(tick_k)       
    ax2.set_yticklabels([])
    ax2.set_xlabel('$k$') 
    ax2.set_ylabel(r'$\rho(k)$')
    ax2.grid(axis='x')
    ax2.set_title('States density as function of $k$')
    ax2.set_xlim(0, n_max+0.5)
    ax2.legend(loc='upper right')
    
    ###### Third subplot #######
    ax3 = fig3.add_subplot(grid[1,0])
    xs4 = np.linspace(0,n_max**2,n_max)
    
    # plot the scatter points
    for i in range(0,n_max+1):
        if (xs3_discrate[i-1])**2 >= xs4[n -1] and (xs3_discrate[i-1])**2 < xs4[n -1] + xs4[1]:
            ax3.scatter((xs3_discrate[i-1])**2, 0, color = 'r', s = 60, zorder=2)
        else:
            ax3.scatter((xs3_discrate[i-1])**2, 0, color = 'b', s = 40, zorder=2)
    
    # ticks for the k axis 
    ax3.set_xticks(np.arange(1,n_max + 1,1)**2)
    tick_e = ax3.get_xticks().tolist()
    for i, tick in enumerate(tick_e):
        tick_e[i] = r'$E_{{ {0} }}$'.format(i+1)

    # plot additional feature of the graph
    ax3.set_xlabel('$\epsilon$') 
    ax3.plot(np.linspace(0, n_max**2*1.05, 50), np.zeros(50), color='k',zorder=1)
    ax3.set_yticklabels([])
    ax3.set_xticklabels(tick_e)
    ax3.grid(axis='x')
    ax3.set_title('States in the energy-space')
    
    
    rectangle = plt.Rectangle((xs4[n -1],-2), xs4[1], 4, alpha = 0.4, color = 'r', zorder=1)
    ax3.add_artist(rectangle)
    
    ###### Fourth subplot #######
    ax4 = fig3.add_subplot(grid[1,1])
    epsilons = np.arange(0.1,n_max*1.05, 0.1)
    g = 1/np.sqrt(epsilons)
    
    ax4.plot(epsilons, g, color='k',zorder=1, label=r'$g(\epsilon) = \frac{1}{\sqrt{\epsilon}}$')
    # ax4.scatter((xs3_discrate[n - 1]), 1/np.sqrt(xs3_discrate[n - 1]) , color = 'r', s = 60, zorder=2,)
    
    
    ax4.set_xticks(np.arange(0,n_max+1, 1))
    tick_e2 = ax4.get_xticks().tolist()
    factor = r'$\; \frac{\hbar^2 \pi^2}{2 m L^2}$'
    for i in range(len(tick_e2)):
        if i == 0:
            tick_k[i] = '0'
        else:
            value = '{}'.format(int(tick_e2[i])**2)
            tick_k[i] = value + factor
   
    # calculate the points of the plot
    ys4 = np.zeros(len(xs4))
    for i in range(len(xs4)):
        ys4[i] = np.sum( np.where((xs3_discrate**2 >= xs4[i]) & (xs3_discrate**2 < xs4[i] +  xs4[1]) , 1, 0))
        
    for i in range(len(xs4)):
        if xs3_discrate[i] == n:
            ax4.scatter(xs3_discrate[i], ys4[i], color = 'r', s = 60, zorder=2,)
        else:
            ax4.scatter(xs3_discrate[i], ys4[i], color = 'b', s = 40, zorder=2,)
            
    ax4.set_xlim(ax2.get_xlim())
    tick_e.insert(0,'0')
    ax4.set_xticklabels(tick_e)
    ax4.grid()
    ax4.set_xlabel('$\epsilon$') 
    ax4.set_ylabel('$g(\epsilon)$')
    ax4.set_title('States density as function of $\epsilon$')
    ax4.legend(loc='upper right')
    
    ###### Fifth subplot #######
    ax5 = fig3.add_subplot(grid[2,:])
    xs3b = np.linspace(0,L,100)
    ys3b = wavefunction1D(xs3b, n, L, 0, alpha)
    ax5.plot(xs3b, ys3b , color='b')
    
    ax5.set_xticks(np.linspace(0,1,5))
    tick_L = ['0',  r'$\frac{L}{4}$',  r'$\frac{L}{2}$', r'$\frac{3}{4} L$','L' ]
    ax5.set_xticklabels(tick_L)
    ax5.set_yticklabels([])
    
    ax5.grid(axis = 'x')
    
    ax5.axhline(y=0, color='k', alpha = 0.8)
    ax5.axvline(x=0, color='r')
    ax5.axvline(x=L, color='r')
    ax5.set_ylim(- np.sqrt(np.pi/L)*1.05, np.sqrt(np.pi/L)*1.05)
    ax5.set_title('Wavefunction')
    ax5.set_ylabel(r'$\psi$')


interactive(children=(IntSlider(value=4, description='n', max=8, min=1), Output()), _dom_classes=('widget-inte…

<font color=red>Note:  $g(\epsilon) = \frac{1}{\sqrt{\epsilon}}$ is closer to the 'data points' respect to $g(\epsilon) = \frac{L}{\pi \sqrt{\epsilon}}$. Same for $\rho(k) = 1$ (closer) and $\rho(k) = \frac{L}{\pi}$

</font> 

## 2) <a id='2'> </a> Density of States


### 2D and 3D Densities of States

For a 2-D Infinite Square Well, the Time Indipendent Schrödinger equation is separable and gives the solutions: 
$$
\psi_{n_x, n_y} (x, y)= \sqrt{\frac{2}{L}} \sin \left(\frac{n_x \pi}{L}x \right) \sin \left(\frac{n_y \pi}{L}y \right)
$$
and energy:
$$
E_n = \frac{\hbar^2 k^2}{2 m} = \frac{\hbar^2 \pi^2}{2 m L^2} \left(n_x^2 + n_y^2 \right)
$$



In [28]:
def wavefunction2D(x,y,n_x,n_y, L):
    """
    Calculate the wavefunction of a particle in a box 2-D
    Inputs:
    x, y      Values of x and y 
    n_x, n_y  Two quantum number
    L         Lenght of the box
    Output: the 2d wavefunction
    """
    return np.sqrt(2/L)*np.sin(np.pi * n_x * x / L)*np.sin(np.pi * n_y * y / L)

In [29]:
L = 1 

xs = np.linspace(0, L, 50)
ys = np.linspace(0, L, 50)
xx, yy = np.meshgrid(xs,ys)

@interact(n_x=(1,5, 1), n_y =(1,5, 1)) # interacting velocity 
def infinitewell2d(n_x, n_y):
    psi3d = wavefunction2D(xx,yy,n_x,n_y, L)

    fig2 = plt.figure(figsize = (10,8))
    ax3d = fig2.add_subplot(111, projection = '3d')
    ax3d.plot_surface(xx,yy,psi3d,cmap='viridis')
    ax3d.view_init(40, 130)
    ax3d.set_xlim(0,L)
    ax3d.set_ylim(0,L)
    ax3d.set_zlim(-np.sqrt(2/L),np.sqrt(2/L))
    ax3d.set_xlabel('x')
    ax3d.set_ylabel('y')
    ax3d.set_zlabel(r'$\psi$')
    #print("the energy of the particle $\frac{\hbar^2 \pi^2}{2 m L^2}$", n_x**2 + n_y**2)

interactive(children=(IntSlider(value=3, description='n_x', max=5, min=1), IntSlider(value=3, description='n_y…

### Density of States in 1D

Given the quantised nature of $k$ (and thus of the energy), the states of a particle in a box represent a series a point in the "k-space" separated by $\Delta k = \frac{\pi}{L}$. *i.e.* every increment of $\Delta k = \frac{\pi}{L}$ contains one k-state. This give the density of states:
$$
\rho (k) = \frac{\pi}{L} 
$$
Since, in 1-D, each k states correspond to an energy state $\epsilon = \frac{\hbar^2 k^2}{2 m}$, then there is one state per $\Delta \epsilon = \Delta k \cdot \frac{\Delta \epsilon}{\Delta k}$, *i.e.*:
$$
\Delta \epsilon \approx \Delta k \cdot \frac{d \epsilon}{d k} = \frac{\pi}{L} \frac{\hbar^2 k}{2 m} \propto \sqrt{\epsilon}
$$
which gives the density of states:
$$
g(\epsilon) = \frac{1}{\Delta \epsilon} \propto \frac{1}{\sqrt{\epsilon}}
$$

In [30]:
n_max = 8
L = 1
alpha = 1

xs3_discrate = np.arange(1,n_max+1,1)
xs3 = np.linspace(0, n_max*1.05, 50)

@interact(n=(1,n_max, 1)) # interacting velocity 
def multiplicity(n):
    fig3 = plt.figure(figsize = (12,8),constrained_layout=True)
    grid = fig3.add_gridspec(3,2)
    
    ###### First subplot #######
    ax1 = fig3.add_subplot(grid[0,0])
    # plot the scatter points
    for i in range(0,n_max+1):
        if i == n:
            ax1.scatter(xs3_discrate[i-1], 0, color = 'r', s = 60, zorder=2)
        else:
            ax1.scatter(xs3_discrate[i-1], 0, color = 'b', s = 40, zorder=2)
    
    # ticks for the k axis 
    ax1.set_xticks(np.arange(0,n_max+1, 1))
    tick_k = ax1.get_xticks().tolist()
    factor = r'$\; \frac{\pi}{L}$' 
    for i in range(len(tick_k)):
        if i == 0:
            tick_k[i] = '0'
        else:
            value = '{}'.format(int(tick_k[i]))
            tick_k[i] = value + factor
    
    # plot additional feature of the graph
    ax1.set_xlabel('$k$') 
    ax1.plot(xs3, np.zeros(50), color='k',zorder=1)
    ax1.set_yticklabels([])
    ax1.set_xticklabels(tick_k)
    ax1.grid()
    ax1.set_title('States in the k-space')
    rectangle = plt.Rectangle((n,-2),1, 4, alpha = 0.4, color = 'r', zorder=1 )
    ax1.add_artist(rectangle)
    ax1.axvline(x=n, zorder=1, color='r')
    ax1.axvline(x=n+1, zorder=1, color='r' )
    
    ###### Second subplot #######
    ax2 = fig3.add_subplot(grid[0,1])
    
    # calculate the points of the plot
    #rs2 = 
    xs2 = np.arange(1, n_max + 1, 1)
    ys2 = np.zeros(len(xs2))
    for i in range(len(xs2)):
        ys2[i] = np.sum(np.where((xs3_discrate >= xs2[i]) & (xs3_discrate < xs2[i]+1) , 1, 0))
        
    for i in range(len(xs2)):
        if xs2[i] == n:
            ax2.scatter(xs2[i], ys2[i], color = 'r', s = 60, zorder=2,)
        else:
            ax2.scatter(xs2[i], ys2[i], color = 'b', s = 40, zorder=2,)
            
    ax2.plot(xs3, np.ones(50), color='k', zorder=1)
    
    # plot additional feature of the graph
    ax2.set_xticks(np.arange(0,n_max+1, 1))
    ax2.set_xticklabels(tick_k)
    tick_y = ax2.get_yticks().tolist()
    for i in range(len(tick_y)):
        if tick_y[i] == 1.0:
            tick_y[i] = '' #r'$\frac{L}{\pi}$'
        else:
            tick_y[i] = ''
            
    ax2.set_yticklabels(tick_y)
    ax2.set_xlabel('$k$') 
    ax2.set_ylabel(r'$\rho(k)$')
    ax2.grid()
    ax2.set_title('States density as function of $k$')
    
    ###### Third subplot #######
    ax3 = fig3.add_subplot(grid[1,0])
    
    # plot the scatter points
    for i in range(0,n_max+1):
        if i == n:
            ax3.scatter((xs3_discrate[i-1])**2, 0, color = 'r', s = 60, zorder=2)
        else:
            ax3.scatter((xs3_discrate[i-1])**2, 0, color = 'b', s = 40, zorder=2)
    
    # ticks for the k axis 
    tick_e = ax3.get_xticks().tolist()
    factor = r'$\; \frac{\hbar^2 \pi^2}{2 m L^2}$'
    for i in range(0,len(tick_e)):
        if i == 1:
            tick_e[i] = '0'
        else:
            value = '{}'.format(int(tick_e[i]))
            tick_e[i] = value + factor
    
    # plot additional feature of the graph
    ax3.set_xlabel('$\epsilon$') 
    ax3.plot(np.linspace(0, n_max**2*1.05, 50), np.zeros(50), color='k',zorder=1)
    ax3.set_yticklabels([])
    ax3.set_xticklabels(tick_e)
    ax3.grid()
    ax3.set_title('States in the energy-space')
    
    ###### Fourth subplot #######
    ax4 = fig3.add_subplot(grid[1,1])
    epsilons = np.arange(0.2,n_max*1.05, 0.1)
    g = 1/np.sqrt(epsilons)
    
    ax4.plot(epsilons, g, color='k',zorder=1)
    ax4.scatter((xs3_discrate[n - 1]), 1/np.sqrt(xs3_discrate[n - 1]) , color = 'r', s = 60, zorder=2,)
    
    
    ax4.set_xticks(np.arange(0,n_max+1, 1))
    tick_e2 = ax4.get_xticks().tolist()
    factor = r'$\; \frac{\hbar^2 \pi^2}{2 m L^2}$'
    for i in range(len(tick_e2)):
        if i == 0:
            tick_k[i] = '0'
        else:
            value = '{}'.format(int(tick_e2[i])**2)
            tick_k[i] = value + factor
            
            
    ax4.set_ylim(0, np.max(g)*1.05)
    ax4.set_xlim(ax2.get_xlim())
    ax4.set_xticklabels(tick_k)
    ax4.set_yticklabels([])
    ax4.grid()
    ax4.set_xlabel('$\epsilon$') 
    ax4.set_ylabel('$g(\epsilon)$')
    ax4.set_title('States density as function of $\epsilon$')
    
    ###### Fifth subplot #######
    ax5 = fig3.add_subplot(grid[2,:])
    xs3b = np.linspace(0,L,100)
    ys3b = wavefunction1D(xs3b, n, L, 0, alpha)
    ax5.plot(xs3b, ys3b , color='r')
    
    ax5.set_xticks(np.linspace(0,1,5))
    tick_L = ['0',  r'$\frac{L}{4}$',  r'$\frac{L}{2}$', r'$\frac{3}{4} L$','L' ]
    ax5.set_xticklabels(tick_L)
    
    ax5.set_yticks(np.linspace(- np.max(ys3b) ,np.max(ys3b),3))
    tick_w = [r'$- \sqrt{\frac{\pi}{L}}$','0', r'$+\sqrt{\frac{\pi}{L}}$']
    ax5.set_yticklabels(tick_w)
    
    ax5.plot(xs3b, np.zeros(100), color ='k')
    ax5.grid()
    ax5.axvline(x=0)
    ax5.axvline(x=L)
    ax5.set_ylim(- np.sqrt(np.pi/L)*1.05, np.sqrt(np.pi/L)*1.05)
    ax5.set_title('Wavefunction')


interactive(children=(IntSlider(value=4, description='n', max=8, min=1), Output()), _dom_classes=('widget-inte…

<font color=red>Note: Are the axes' values vorrect?

</font> 


### Density of States in 2D and 3D
In 2D and 3D, we can determin $\rho (k)$ and $g(\epsilon)$ in a similar way, knowing that $\epsilon = \frac{\hbar^2 k^2}{2 m}$, with $k$ defined as $\left| \vec{k} \right| = \sqrt{k_x^2 + k_y^2}$ in 2D and $\left| \vec{k} \right| = \sqrt{k_x^2 + k_y^2 + k_z^2}$ in 3D. $\rho (k)$ is the number of states with the wave nubr between $k$ and $k + \Delta k$, and $g(\epsilon) = \rho (k) \frac{\Delta k}{\Delta \epsilon} \approx \rho (k) \frac{d k}{d \epsilon} $.
In 2D, $\rho (k)$ is the number of states which lie in the circular shell with inner radius $k$ and outer radius $k + \Delta k$, in the k-space. Similarly, in 3D, $\rho (k)$ is the number of states which lie in the spherical shell with inner radius $k$ and outer radius $k + \Delta k$, in the k-space. This implies that the densities scale with the area of the circular shell and volume of the spherical shell, for 2D and 3D space respectively.

Note: In order to simplify the labels, in the for the following simulation $L=\pi$ and $\hbar = m = 1$.

In [31]:
kmax = 10                  # Max velocity
dk = 1                    # Delta velocity
Npoints_2D = (kmax)**2    # Number of lattice points
Npoints_3D = (kmax)**3    # Number of lattice points

# generate lattice points 2D
rbox_2D = np.zeros((Npoints_2D,2))
count = 0
for i in range(kmax): # iterate over k_x, k_y
    for j in range(kmax):
        rbox_2D[count] = np.array([i,j])*dk # save the position of the lattice point
        count += 1 # move to the next point
        
# generate lattice points 3D
rbox_3D = np.zeros((Npoints_3D,3))### a) 2-D Simulation
count = 0
for i in range(kmax): # iterate over v_x, v_y, v_z
    for j in range(kmax):
        for k in range(kmax):
            rbox_3D[count] = np.array([i,j,k])*dk # save the position of the lattice point
            count += 1 # move to the next point
            

# generate angles for sphere
phi = np.linspace(0, np.pi/2, 20)
theta = np.linspace(0, np.pi/2, 20)
    
# find points of the unit sphere
x3d = np.outer(np.cos(phi), np.sin(theta))
y3d = np.outer(np.sin(phi), np.sin(theta))
z3d = np.outer(np.ones(np.size(phi)), np.cos(theta))

ks_continuos = np.linspace(0, kmax, 50)
epsilon_continuos = np.linspace(0, kmax**2, 50)

# calculate the distance (k)
ks_2D = np.zeros(Npoints_2D)
for i in range(Npoints_2D):
    ks_2D[i] = np.linalg.norm(rbox_2D[i])
    
ks_3D = np.zeros(Npoints_3D)
for i in range(Npoints_3D):
    ks_3D[i] = np.linalg.norm(rbox_3D[i])

# calulate the number of states for k and epsilon (discrate)
xs_2D = np.arange(1, kmax , dk)
ys_2D_k = np.zeros(len(xs_2D))
ys_2D_e = np.zeros(len(xs_2D))
for i in range(len(xs_2D)):
    ys_2D_k[i] = np.sum(np.where((ks_2D >= xs_2D[i]) & (ks_2D < xs_2D[i]+dk) , 1, 0))
    ys_2D_e[i] = np.sum(np.where((ks_2D**2 >= (xs_2D[i])**2) & (ks_2D**2 < (xs_2D[i]+dk)**2) , 1, 0))
    
xs_3D = np.arange(1, kmax , dk)
ys_3D_k = np.zeros(len(xs_3D))
ys_3D_e = np.zeros(len(xs_3D))
for i in range(len(xs_3D)):
    ys_3D_k[i] = np.sum(np.where((ks_3D >= xs_3D[i]) & (ks_3D < xs_3D[i]+dk) , 1, 0))
    ys_3D_e[i] = np.sum(np.where((ks_3D**2 >= (xs_3D[i])**2) & (ks_3D**2 < (xs_3D[i]+dk)**2) , 1, 0))
    



In [32]:
@interact(k=(1,kmax - 1, dk)) # interacting velocity 
def kspace(k):
    fig4 = plt.figure(figsize = (14,11))
    grid = fig4.add_gridspec(4,4,  wspace=0.3, hspace=0.5)
    
    ###### First subplot #######
    ax1 = fig4.add_subplot(grid[0:2,0:2])
    
    # plot the circles 
    circle_out = plt.Circle((0,0), k+ dk, color='r', alpha = 0.4, zorder=1)     # outer circle
    circle_in = plt.Circle((0,0), k, color='w', alpha = 1, zorder=2)            # inner circle
    rectangle_1 = plt.Rectangle((-0.48,-0.5), 0.48, kmax - 0.5, alpha = 1, color = 'w', zorder=3 )
    rectangle_2 = plt.Rectangle((-0.48,-0.5), kmax - 0.5, 0.5, alpha = 1, color = 'w', zorder=3 )
    ax1.add_artist(circle_out)
    ax1.add_artist(circle_in)
    ax1.add_artist(rectangle_1)
    ax1.add_artist(rectangle_2)
  
    # plot the lattice points
    for i in range(Npoints_2D):
        if k  <= ks_2D[i] and ks_2D[i] < k + dk : # if they are inside the shell 
            ax1.scatter(rbox_2D[i,0],rbox_2D[i,1], s = 30, color ='r', zorder=5) # plot points 
        else:
            ax1.scatter(rbox_2D[i,0],rbox_2D[i,1], s = 15, color ='b', zorder=5) # plot points in different color
    
    # add labels, title, axes, grid 
    ax1.set_xlabel('$k_x$') 
    ax1.set_ylabel('$k_y$')
    ax1.set_xlim( - 0.5 , kmax - 0.5)
    ax1.set_ylim(- 0.5 , kmax - 0.5)
    ax1.set_axisbelow(False)
    ax1.set_aspect(aspect = 'equal')
    ax1.axvline(x=0, color='k',zorder=4)
    ax1.axhline(y=0, color='k',zorder=4)
    ax1.set_title('2-D Lattice')
    
    ###### Second subplot #######
    ax2 = fig4.add_subplot(grid[0,2:4])
        
    for i in range(len(xs_2D)):
        if xs_2D[i] == k:
            ax2.scatter(xs_2D[i], ys_2D_k[i], color = 'r', s = 60, zorder=2,)
        else:
            ax2.scatter(xs_2D[i], ys_2D_k[i], color = 'b', s = 40, zorder=2,)
     
    ax2.set_title('2D density of states')
    ax2.set_xlabel(r'$k$')
    ax2.set_ylabel(r'$\rho(k)$')
    ax2.set_yticklabels([])
    ax2.plot(ks_continuos, ks_continuos*np.pi/2, color = 'k', zorder=1 , label = r'$\frac{\pi}{2} dk \; k $')
    ax2.legend(loc = 'upper left')
    
    ###### Third subplot #######
    ax3 = fig4.add_subplot(grid[1,2:4])
    
    for i in range(len(xs_2D)):
        if xs_2D[i] == k:
            ax3.scatter(xs_2D[i]**2, ys_2D_e[i], color = 'r', s = 60, zorder=2,)
        else:
            ax3.scatter(xs_2D[i]**2, ys_2D_e[i], color = 'b', s = 40, zorder=2,)
            
    ax3.set_xlabel(r'$\epsilon$')
    ax3.set_ylabel(r'$g(\epsilon)$')
    ax3.set_yticklabels([])
    ax3.plot(epsilon_continuos, np.pi/2*np.ones(len(epsilon_continuos)), color = 'k', 
             zorder=1, label = r'$\frac{\pi}{2} \frac{m}{\hbar^2}$') 
    
    ax3.legend(loc = 'upper left')
    
    ###### Fourth subplot #######
    ax4 = fig4.add_subplot(grid[2:4,0:2], projection = '3d')
    
    # plot the two spheres
    ax4.plot_surface(k*x3d, k*y3d, k*z3d, color='r', alpha = 0.2)                # inner sphere
    ax4.plot_surface((k+dk)*x3d, (k+dk)*y3d, (k+dk)*z3d, color='r', alpha = 0.2) # outer sphere
    
    # plot the lattice points
    for i in range(Npoints_3D):
        if k  <= ks_3D[i] and ks_3D[i] < k + dk : # if they are inside the shell (-0.05 to include zero)
            ax4.scatter(rbox_3D[i,0],rbox_3D[i,1],rbox_3D[i,2], s = 20, color ='r') # plot points 
        else:
            ax4.scatter(rbox_3D[i,0],rbox_3D[i,1],rbox_3D[i,2], s = 10, color ='b') # plot points in different color
            
    # add labels, title, axes
    ax4.set_xlabel('$k_x$') 
    ax4.set_ylabel('$k_y$')
    ax4.set_zlabel('$k_z$')
    ax4.plot([rbox_3D[:,0].min() - 1, rbox_3D[:,0].max() + 1], [0,0], [0,0], color = 'black')
    ax4.plot([0, 0], [rbox_3D[:,1].min() - 1, rbox_3D[:,1].max() + 1], [0, 0], color = 'black')
    ax4.plot([0, 0], [0,0], [rbox_3D[:,2].min() - 1, rbox_3D[:,2].max() + 1], color = 'black')
    ax4.set_xlim(- 0.5 ,kmax - 0.5)
    ax4.set_ylim(- 0.5 ,kmax - 0.5)
    ax4.set_zlim(- 0.5 ,kmax -0.5)
    ax4.set_title('3-D Lattice')
    
    ###### Fifth subplot #######
    ax5 = fig4.add_subplot(grid[2,2:4])
    
    for i in range(len(xs_3D)):
        if xs_3D[i] == k:
            ax5.scatter(xs_3D[i], ys_3D_k[i], color = 'r', s = 60, zorder=2,)
        else:
            ax5.scatter(xs_3D[i], ys_3D_k[i], color = 'b', s = 40, zorder=2,)
    
    ax5.set_title('3D density of states')
    ax5.set_xlabel(r'$k$')
    ax5.set_ylabel(r'$\rho(k)$')
    ax5.set_yticklabels([])
    ax5.plot(ks_continuos, ks_continuos**2*np.pi/(4*dk**2), color = 'k', zorder=1 , label = r'$\frac{\pi}{2} dk \; k^2 $') 
    #ax5.scatter(k, k**2*np.pi/2, color = 'r', s = 40, zorder=2)
    ax5.legend(loc = 'upper left')
    
    ###### Sixth subplot #######
    ax6 = fig4.add_subplot(grid[3,2:4])
    
    for i in range(len(xs_2D)):
        if xs_2D[i] == k:
            ax6.scatter(xs_3D[i]**2, ys_3D_e[i], color = 'r', s = 60, zorder=2,)
        else:
            ax6.scatter(xs_3D[i]**2, ys_3D_e[i], color = 'b', s = 40, zorder=2,)
            
    
    ax6.set_xlabel(r'$\epsilon$')
    ax6.set_ylabel(r'$g(\epsilon)$')
    ax6.set_yticklabels([])
    ax6.plot(epsilon_continuos, np.sqrt(epsilon_continuos), color = 'k', 
             zorder=1 ,label = r'$\frac{\pi}{2} \frac{m}{\hbar^2}$')
    ax6.scatter(k**2, k, color = 'r', s = 40, zorder=2)
    ax6.legend(loc = 'upper left')

interactive(children=(IntSlider(value=5, description='k', max=9, min=1), Output()), _dom_classes=('widget-inte…

<font color=red>Note: I have plotted the analytic solution as the density of state (not counting the one in the graph, as I did in the NoteBook of Maxwell-Boltzmann distribution). Let me know if I should change it similar to the one in the other notebook. I also have to exclude the case with $k_x = k_y (= k_z) = 0$, which has $\psi = 0$
</font> 


## 3) <a id='3'> </a> Partition Function 

Once we know $g(\epsilon)$, we can approximate the partition function as: 

$$
Z_1 = \sum_{\epsilon} \Omega(\epsilon) e^{- \beta \epsilon} \approx \int_{0}^{\infty} g(\epsilon)e^{- \beta \epsilon} d \epsilon
$$
where we note that $ g(\epsilon) d \epsilon$ is the number of states with energy between $\epsilon$ and $\epsilon + d \epsilon$. With the partition function, we can calculate the other thermodynamic variables.


In [33]:
Tmax = 4
Ts = np.linspace(0, Tmax, 100)
m_max =10

@interact(m=(1, m_max, 1)) # interacting velocity 
def plot_distribution(m):
    fig5 = plt.figure(figsize = (10,7)) # initialise the figure and set the dimension of the plot
    ax = fig5.add_subplot(111)
    
    ax.plot(Ts, np.sqrt(m*Ts), color = 'r', label=r'1D: $\;\; Z_1 = L\left(\frac{2 \pi m k T}{\hbar}\right)^{1/2}$')
    ax.plot(Ts, m*Ts, color = 'b', label=r'2D: $\;\; Z_1 = A\left(\frac{2 \pi m k T}{\hbar}\right)^{2/2}$')
    ax.plot(Ts, (m*Ts)**(3/2), color = 'k', label=r'3D: $\;\; Z_1 = V\left(\frac{2 \pi m k T}{\hbar}\right)^{3/2}$')
    
    ax.axvline(x=0, zorder=1, color='k')
    ax.plot(np.linspace(-1, Tmax, 100), np.zeros(np.size(Ts)),zorder=1, color='k')
    
    ax.legend(loc='upper left')
    ax.set_xlabel('Tempertature (K)')
    ax.set_ylabel(r'$Z_1$')
    ax.set_xlim(-0.3, Tmax + 0.2)
    ax.set_ylim(-1, 20)
    

interactive(children=(IntSlider(value=5, description='m', max=10, min=1), Output()), _dom_classes=('widget-int…